Implementations

In [ ]:
from implementations import *
from helpers import *
import matplotlib.pyplot as plt
import numpy as np

# 1- Load data

In [ ]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data('dataset_to_release/', sub_sample=False)

In [ ]:
print('Shapes of x_train :', x_train.shape)
print('Shapes of y_train :', y_train.shape)
print('Shapes of x_test :', x_test.shape)

# 2- Clean data

standardization

In [ ]:
def standardize_(x):
    """Standardize the original data set."""
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    # Vérifiez si std_x est nul (zéro) avant de diviser
    non_zero_std_indices = std_x != 0
    x[:, non_zero_std_indices] = x[:, non_zero_std_indices] / std_x[non_zero_std_indices]
    return x, non_zero_std_indices

In [ ]:
#Enlever premiere colonne x_train car rempli de nan
x_train = x_train[:,0:]

#Remplacer les nan par la mediane de chaque colonne
median = np.nanmedian(x_train, axis = 0)
x_train_filled = x_train
for col in range(x_train.shape[1]):
    x_train_filled[:,col] = np.nan_to_num(x_train[:,col], nan= median[col])

x_train_standardized, col_to_keep = standardize_(x_train_filled)

#Enlever les colonnes qui ont une std = 0
x_train_cleaned = x_train_standardized[:, col_to_keep]
print(x_train_cleaned.shape)

Correlation Matrix

In [ ]:
correlation_matrix = np.corrcoef(x_train_cleaned, rowvar = False)
plt.figure(figsize=(3, 3))
plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.show()

In [ ]:
#Remplacer la partie triangulaire basse de la matrice par des 0
m = np.tril(np.ones(correlation_matrix.shape), k = 0).astype(bool)
correlation_matrix[m] = 0

Similar columns deletion

In [ ]:
#Calcul des colonnes qui se ressemblent au dessus d'un threshold
threshold = 0.8

#Creation of a matrix True/False where True == When the correlation is above the treshold
mask = np.logical_and(np.abs(correlation_matrix) >= threshold, np.abs(correlation_matrix) < 1)
correlated_features = np.where(mask == True)

#indices_correlated has the indices of the correlated columns
indices_correlated = np.unique(correlated_features[0])

#Removes the correlated columns to x_train_cleaned
x_train_done = np.delete(x_train_cleaned, indices_correlated, axis = 1)
print(x_train_done.shape)

# 3- Split data

In [ ]:
#We split x_train in a train part and a validation part
seed = 12
ratio = 0.8

# set seed
np.random.seed(seed)

#Number of elements in y
number = y_train.shape[0]

index = np.random.permutation(number)

#Separate into 2 categories at the split
split = int(np.floor(ratio * len(x_train_done)))
train = index[:split]
test = index[split:]

#create the new datasets
x_tr = x_train_done[train]
y_tr = y_train[train]
x_te = x_train_done[test]
y_te = y_train[test]

# 4- First step w

### a) Least Squares

In [ ]:
'''
w_initial, loss = least_squares(y_train, x_train_final)
'''

### b) Ridge Regression

In [ ]:
'''
lambda_ = 0.1
w_initial, loss = ridge_regression(y_train, new_data, lambda_)
'''

### c) Looking for good hyperparameters  lambda_ for ridge regression

In [ ]:
lambdas = np.linspace(0,0.5,50)
weights = []
losses_te = []

for lambda_ in lambdas:
    w,_ = ridge_regression(y_tr, x_tr, lambda_)
    weights.append(w)
    loss = compute_loss(y_te, x_te, w)
    losses_te.append(loss)

In [ ]:
best_index = losses_te.index(min(losses_te))
print('Smallest loss : ',losses_te[best_index])
print('Best lambda : ',lambdas[best_index])
plt.figure(figsize = (3,3))
plt.xlabel('lambdas')
plt.ylabel('losses')
plt.plot(lambdas, losses_te)
plt.show()

In [ ]:
w_initial = weights[best_index]

# 4- Gradient descent

### a) With Logistic Regression

In [ ]:
'''
nb_steps = 150
gamma = 0.01

for i in range(nb_steps):
    w_final, loss= logistic_regression(y_train, x_train_final, w_initial, gamma)
'''

### b) With Regularized Logistic Regression

In [ ]:
'''
nb_steps = 150
gamma = 0.01

for i in range(nb_steps):
    w_final, loss= reg_logistic_regression(y_train, new_data, w_initial, gamma, lambda_)
'''

### c) With search of best hyperprameters lambda and gamma for ridge regression AND regularized logistic regression

In [ ]:
nb_steps = 20
lambdas = np.linspace(0,0.4,10)
gammas = np.linspace(0,1,10)
weights = np.zeros((len(lambdas),len(gammas), x_tr.shape[1]))
losses_te = np.zeros((len(lambdas),len(gammas)))

for i, lambda_ in enumerate(lambdas):
    
    for j, gamma in enumerate(gammas):
        
        w, _ = ridge_regression(y_tr, x_tr, lambda_)

        for step in range(nb_steps):

            loss, w = reg_logistic_regression(y_tr, x_tr, w, gamma, lambda_)

        weights[i,j] = w
        loss_te = compute_loss(y_te, x_te, w)
        losses_te[i][j] = loss_te           

In [ ]:
best_index = np.unravel_index(np.argmin(losses_te), losses_te.shape)
row_index, col_index = best_index
best_weight = weights[row_index][col_index]
print('Best lambda : ', lambdas[row_index])
print('Best gamma : ', gammas[col_index])
w_final = best_weight

# 5- Test

Prepare x_test

In [ ]:
x_test = x_test[:,0:]

#Remplacer les nan par la mediane de chaque colonne
median = np.nanmedian(x_test, axis = 0)
x_test_filled = x_test
for col in range(x_test.shape[1]):
    x_test_filled[:,col] = np.nan_to_num(x_test[:,col], nan= median[col])

x_test_standardized, _ = standardize_(x_test_filled)

#Enlever les colonnes qui ont une std = 0
x_test_cleaned = x_test_standardized[:, col_to_keep]


x_test_final = np.delete(x_test_cleaned, indices_correlated, axis = 1)

print(x_test_final.shape)

Finding y predicted with x_test and w

In [ ]:
#y_pred_proba has the proba of each y to be 1 or -1
y_pred_proba = sigmoid (np.dot(x_test_final, w_final))

#If proba > 0.5, y becomes 1, else -1
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)
y_pred[y_pred == 0] = -1

# 6- Creating the submission file

In [ ]:
name = 'test_6.csv '
create_csv_submission(test_ids, y_pred, name)